## Test data

In [17]:
test_ra = 168.3700 #degrees
test_dec = 22.15167 #degrees
test_d = 233 #kpc
mu_a = -6.930000 #mas/cent
mu_d = -8.670000 #mas/cent
rv = 79.1 #km/s

## Python Functions to Get Positions and Velocities

In [223]:
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord, Galactocentric, ICRS, Galactic
import gala.coordinates as gc
    
#Constants
asec_to_rad = 206264.8 #Converts arcseconds to radians
year_to_sec = 3.155693e7 #Converts from years to seconds
pc_to_km = 3.08568e13 #Converts from pc to km
year_to_day = 365.2422 #Converts years to days
ra_lsr = 318.00425 #RA of the apex of the LSR motion (ie., l=90, b=0)     
dec_lsr = 48.329611 #DEC of the apex of the LSR motion (J2000.0)
ra_sa = 251.51 #RA of the apex of the solar peculiar motion (wrt LSR) J2000.0 
dec_sa = 10.129 #DEC of the apex of the solar peculiar motion (wrt LSR)
alpha_NGP = 192.85948 #RA in degrees of the Galactic Pole (J2000.0)
delta_NGP = 27.12825 #DEC in degrees of the Galactic Pole (J2000.0)
gc_GAN = 32.93192 #Angle along galactic equator from (l,b)=(0,0) to galactic ascending node
l_NCP = 122.9319#Galactic longitude of the NCP (J2000.0)
b_NCP =  27.1283#Galactic latitude of the NCP  (J2000.0)
usun = -10.0 #U-component of Sun's velocity w.r.t. LSR; Dehnen & Binney                  
vsun = 11.0 #V-component of Sun's velocity w.r.t. LSR; 1998, MNRAS, 
wsun = 7.0 #W-component of Sun's velocity w.r.t. LSR; 298, 387
vlsr = 237.0 #Circular velocity of LSR w.r.t. Galaxy center in km/s
dlsr = 8.2 #Distance of LSR from the Galaxy center in pc

def sin_deg(x):
    #Sine function for degree-valued args
    deg_to_rad = 1.74532925199e-02
    return np.sin(np.deg2rad(x))

def cos_deg(x):
    #Cosine function for degree-valued args
    return np.cos(np.deg2rad(x))

def asin_deg(x):
    #Gives asin in degrees
    return np.rad2deg(np.arcsin(x))

def acos_deg(x):
    #Gives acos in degrees
    return np.rad2deg(np.arccos(x))

def atan2_deg(y,x):
    #Gives atan2 in degrees
    return np.rad2deg(np.arctan2(y,x))

def radec_to_lb(ra, dec):
    #Returns dictionary of galactic coords (l, b) in degrees, given an object's RA/DEC in degrees
    coords = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
    galactic = coords.galactic
    return galactic.l.deg, galactic.b.deg

def lb_to_galactic_xyz(l, b, d, dlsr):
    #Takes l and b in degrees, d (distance to object) in kpc, dlsr in kpc
    x = (dlsr-d*cos_deg(b)*cos_deg(l))
    y = -d*cos_deg(b)*sin_deg(l)
    z = d*sin_deg(b)
    return x, y, z #kpc

def pm_radec_to_lb(ra, dec, dist, mu_a, mu_d, rv):
    #Takes ra/dec in degrees, distance in pc, mu_a/mu_d in mas/cent, rv in km/s. Returns mu_l/mu_b in mas/cent
    c = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, distance=dist*u.pc)
    pm = [mu_a, mu_d] * u.mas/(100*u.yr) 
    
    result = gc.pm_icrs_to_gal(c, pm).to(u.mas/(u.yr))*u.yr/u.mas
    mu_l = result[0]*100
    mu_b = result[1]*100
    return mu_l, mu_b

def GETVT(mux,muy,d,asec_to_rad=asec_to_rad,year_to_sec=year_to_sec,pc_to_km=pc_to_km):
    #mux, muy in mas/cent, distance in kpc
    #Get into arcsec/year
    mux = (mux*(1e-3)/100)
    muy = (muy*(1e-3)/100)

    vtx=mux*d*1000*pc_to_km/(year_to_sec*asec_to_rad)
    vty=muy*d*1000*pc_to_km/(year_to_sec*asec_to_rad)
    vt=np.sqrt(vtx*vtx+vty*vty)
    return vtx, vty, vt  

def get_v_wrt_lsr(l, b, vtl, vtb, v_r_sun, usun, vsun, wsun):
    #Finds velocity of galaxy wrt LSR. velocity inputs are in km/s
    
    #Theta direction
    vg = v_r_sun*cos_deg(b)*sin_deg(l) - vtb*sin_deg(b)*sin_deg(l)+vtl*cos_deg(l) + vsun
    #Pi direction
    ug = -v_r_sun*cos_deg(b)*cos_deg(l) + vtb*sin_deg(b)*cos_deg(l) + vtl*sin_deg(l) + usun
    #Z direction
    wg =  v_r_sun*sin_deg(b) + vtb*cos_deg(b) + wsun
    
    return ug, vg, wg #km/s

def get_v_cyl(l,b,d,ug,vg,wg, dlsr=dlsr, vlsr=vlsr):
    #input velocities in km/s, distainces in kpc
    
    x=np.sqrt(dlsr*dlsr+d*d*cos_deg(b)*cos_deg(b)-2.0*dlsr*d*cos_deg(b)*cos_deg(l))
    sinalpha=d*cos_deg(b)*sin_deg(l)/x
    cosalpha=(x*x+dlsr*dlsr-d*d*cos_deg(b)*cos_deg(b))/(2.0*x*dlsr)

    vpi =  ug*cosalpha + (vg+vlsr)*sinalpha
    vtheta = -ug*sinalpha + (vg+vlsr)*cosalpha
    vz =  wg
    
    #upg(2) =  sqrt((ug(2)*cosalpha)**2 + (vg(2)*sinalpha)**2)
    #vpg(2) =  sqrt((ug(2)*sinalpha)**2 + (vg(2)*cosalpha)**2)
    #wpg(2) =  wg(2)
    
    return vpi, vtheta, vz #km/s

def get_v_xyz(l, b, d, vpi, vth, vz, dlsr=dlsr):
    #distances in kpc, vels in km/s

    x=dlsr-d*cos_deg(b)*cos_deg(l)
    y=-d*cos_deg(b)*sin_deg(l)
    z=d*sin_deg(b)

    costh=x/np.sqrt(x**2+y**2)
    sinth=-y/np.sqrt(x**2+y**2)
    vx=vpi*costh-vth*sinth
    vy=-vpi*sinth-vth*costh
    
    return vx, vy, vz

def get_v_spherical(l, b, d, vpi, vtheta, vz, dlsr=dlsr):
#                 --- The transformation equations: ---
#       uppg =  upg*cos(beta) + wpg*sin(beta)
#       vppg =  vpg
#       wppg = -upg*sin(beta) + wpg*cos(beta)
#
#     --- beta is an angle between the line-of-sight to a galaxy and
#         line x (see subroutine GETUPGVPGWPG for definition).
#     --- From the law of cosines...

    x=np.sqrt(dlsr*dlsr+d*d*cos_deg(b)*cos_deg(b)-2.0*dlsr*d*cos_deg(b)*cos_deg(l))
    cosbeta=x/np.sqrt(x*x+d*d*sin_deg(b)*sin_deg(b))
    sinbeta=d*sin_deg(b)/np.sqrt(x*x+d*d*sin_deg(b)*sin_deg(b))
    vr =  vpi*cosbeta + vz*sinbeta
    vrot = vtheta
    v_perp_radial = -vpi*sinbeta + vz*cosbeta

    vt = np.sqrt(vrot**2+v_perp_radial**2)
    
    return vr, vrot, v_perp_radial, vt

## Print Values

In [87]:
l, b = radec_to_lb(test_ra, test_dec)
print "l = {0}, b = {1}" .format(l,b) +" (deg)" + "\n"

x0, y0, z0 = [i for i in lb_to_galactic_xyz(l, b, test_d, dlsr)]
print "Galactocentric X, Y, Z = {0}, {1}, {2}" .format(x0, y0, z0) + " (kpc)" + "\n"

mu_l, mu_b = pm_radec_to_lb(test_ra, test_dec, test_d, mu_a, mu_d, rv) #*u.yr/u.mas
print "mu_l = {0}, mu_b = {1}" .format (mu_l, mu_b) + " (mas/cent)" + "\n"

vtl, vtb, vt = GETVT(mu_l, mu_b, test_d)
print "v_tang in l direction = {0}, v_tang in b direction = {1}, v_tang total = {2}".format(vtl, vtb, vt) \
+ " (km/s)" + "\n"

ug, vg, wg = get_v_wrt_lsr(l, b, vtl, vtb, rv, usun, vsun, wsun)
print "Velocities of galaxy wrt LSR:"
print "u = {0}, v = {1}, w = {2}" .format(ug, vg, wg) + " (km/s)" + "\n"

vpi, vtheta, vz = get_v_cyl(l,b,test_d,ug,vg,wg)
print "Cylindrical velocities:"
print "vpi = {0}, vtheta = {1}, vz = {2}" .format(vpi, vtheta, vz) + " (km/s)" + "\n"

vx0, vy0, vz0 = get_v_xyz(l, b, test_d, vpi, vtheta, vz)
print "Galactocentric XYZ velocities:"
print "Vx = {0}, Vy = {1}, Vz = {2}" .format (vx0, vy0, vz0) + " (km/s)" + "\n"

vr, vrot, v_perp_radial, vt = get_v_spherical(l, b, test_d, vpi, vtheta, vz)
print "Radial velocity = {0}" .format(vr) + " (km/s)"
print "Velocity in direction of disk rotation = {0}" .format(vrot) + " (km/s)"
print "V up and perp to radial = {0}" .format(v_perp_radial) + " (km/s)"
print "Tangential velocity = {0}" .format(vt) + " (km/s)"

l = 220.16913737, b = 67.2312443585 (deg)

Galactocentric X, Y, Z = 77.1058750111, 58.1663853521, 214.843319663 (kpc)

mu_l = 6.17081529568, mu_b = -9.22577180982 (mas/cent)

v_tang in l direction = 68.1599852049, v_tang in b direction = -101.903628602, v_tang total = 122.597443307 (km/s)

Velocities of galaxy wrt LSR:
u = 41.2271830735, v = -121.440923417, w = 40.498068962 (km/s)

Cylindrical velocities:
vpi = -36.6806351847, vtheta = 117.081645903, vz = 40.498068962 (km/s)

Galactocentric XYZ velocities:
Vx = 41.2271830735, Vy = -115.559076583, Vz = 40.498068962 (km/s)

Radial velocity = 21.896955667 (km/s)
Velocity in direction of disk rotation = 117.081645903 (km/s)
V up and perp to radial = 50.0608222039 (km/s)
Tangential velocity = 127.334982338 (km/s)


# Orbit

In [145]:
#Constants
G = 4.498933261e-6 #Gravitational constant in kpc^3/(M_sun Gyr)
vcon = 1.0226903 #Conversion factor from kpc/Gyr to km/s (i.e. multiply by km/s to get kpc/Gyr)

print l, b, test_d
print x0, y0, z0
print vx0, vy0, vz0


#Initial angular momenta
amx0=y0*vz0-z0*vy0
amy0=z0*vx0-x0*vz0
amz0=x0*vy0-y0*vx0

print amx0, amy0, amz0

NMC = 1000 #number of MC experiments


220.16913737 67.2312443585 233
77.1058750111 58.1663853521 214.843319663
41.2271830735 -115.559076583 40.498068962
27182.7219156 5734.74582831 -11308.3199331


## XENERGY, XFORCE

In [92]:
def XENERGY(x, y, z, vx, vy, vz, vcon=vcon, G=G):
    #bulge parameters
    mb=9.1e9
    rc=2.1
    
    #disk parameters
    #increase disk mass to get higher v_c at R=R_0.
    
    a=4.9e0
    b=0.30e0
    md=1.35e11
    
    #halo parameters
    mv=1.16e12
    c=10.0
    ra=28.2

    #The kinetic energy per unit mass
    ke=vcon*vcon*(vx*vx+vy*vy+vz*vz)/2.0
    
    #The potential energy per unit mass
    r  = np.sqrt(x*x+y*y+z*z)
    rd2 = x*x+y*y
    
    #disk
    phi1=-G*md/np.sqrt(rd2+(a+np.sqrt(z*z+b*b))**2)

    #bulge; this isn't right, but is accurate for r >> rc and that
    #is the region of interest for all or almost all of our range of
    #integration
    
    if r > rc/2.0:
        phi2 = -G*mb/r
    else:
        phi2 = -G*2.0*mb/rc
        
    #halo
    phi3 = -G*(mv/(np.log(1.0+c)-c/(1.0+c))) * np.log(1.0+r/ra)/r

    e=ke+phi1+phi2+phi3

    return ke,phi1,phi2,phi3,e

ke, phi1, phi2, phi3, e = XENERGY(x0, y0, z0, vx0, vy0, vz0)
print ke, phi1, phi2, phi3, e

8729.91904415 -2530.30156307 -173.803322552 -33269.8254141 -27244.0112556


In [239]:
def gammq2(a, x):
    import scipy.special
    #Upper incomplete gamma function i.e. 1 - gamma
    #return scipy.special.gammaincc(a,x)
    return scipy.special.gammaincc(a,x)
print gammq2(1,2)



def gammq(a,x):
    #a-1 is power inside the integral
    #x = lower bound of integration
    #mpmath.gammainc defines them differently,
    #so we make the switch when calling it herein
    import mpmath
    mpmath.mp.dps = 15 # arbitrary precision!
    return mpmath.gammainc(z=a, a=x, regularized=True)
print gammq(1,2)

0.135335283237
0.135335283236613


In [246]:
def XFORCE(x, y, z, G=G):

    #bulge parameters
    mb=9.1e9
    rc=2.1
    #increase disk mass to get higher v_c at R=R_0.
    a=4.9
    b=0.30
    md=1.35e11
    #halo parameters
    mv=1.16e12
    c=10.0
    ra=28.2
    
    r  = np.sqrt(x*x+y*y+z*z)
    rd2 = x*x+y*y
    
    #bulge radial force: -GM_b(r)/r*r
    mb_r = mb*(1.-1.0891*np.exp(-r/rc)*(r/rc)**.2 - gammq(.2, r/rc))
    #mb_r=mb*(1.00 - 1.0891*np.exp(-r/rc)*(r/rc)**.2 - gammq(.2,r/rc))
    fb_r = -G*mb_r/(r*r)

    #NFW halo radial force: -GM_nfw(r)/r*r
    mnfw_r=mv*(np.log(1.0+r/ra)-r/(ra+r))/(np.log(1.0+c)-c/(1.0+c))
    fh_r = -G*mnfw_r/(r*r)

    #Find the X components of the force.
    fx1=-G*md*x/(np.sqrt(rd2+(a+np.sqrt(z*z+b*b))**2))**3
    fx2=x*fb_r/r
    fx3=x*fh_r/r
    fx=fx1+fx2+fx3

    #Find the Y components of the force.
    fy1=-G*md*y/(np.sqrt(rd2+(a+np.sqrt(z*z+b*b))**2))**3
    fy2=y*fb_r/r
    fy3=y*fh_r/r
    fy=fy1+fy2+fy3
    
    #Find the Z components of the force.
    fz1=-G*md*z/(np.sqrt(rd2+(a+np.sqrt(z*z+b*b))**2))**3
    fz1=fz1*(1.0+a/np.sqrt(z*z+b*b))
    fz2=z*fb_r/r
    fz3=z*fh_r/r
    fz=fz1+fz2+fz3

    return fx, fy, fz

print XFORCE(x0, y0, z0)
fx, fy, fz = XFORCE(x0, y0, z0)

(mpf('-31.39233213980949'), mpf('-23.681444352731319'), mpf('-87.684957932643584'))


In [247]:
NSTEP = 1000. #Number of integration steps
t = 5. #total integration time in Gyr
dt = t/NSTEP #timestep interval size in GYr
print dt

0.005


# Integrate orbit

In [248]:
NSTEP = 100. #Number of integration steps
t = 5. #total integration time in Gyr
dt = t/NSTEP #timestep interval size in Gyr

def integrate_orbit(pot, x=x0, y=y0, z=z0, vx=vx0, vy=vy0, vz=vz0, vcon=vcon, dt=dt, NSTEP=NSTEP):
    
    timestep = 0.0
    time = [timestep]
    r2 = [x*x+y*y+z*z]
    xt = [x]
    yt = [y]
    zt = [z]
#    if pot == 'X':
 #       fx, fy, fz = XFORCE(x,y,z)
    for i in np.arange(NSTEP-1):
        if pot == 'X':
            fx, fy, fz = XFORCE(x,y,z)
        vx=vx+0.5*dt*fx/vcon
        vy=vy+0.5*dt*fy/vcon
        vz=vz+0.5*dt*fz/vcon
        x=x+dt*vx*vcon
        y=y+dt*vy*vcon
        z=z+dt*vz*vcon
        r2=x*x+y*y+z*z
        timestep = timestep + dt
        print type(r2)
        time.append(timestep)
        #xt.append(x)
        #yt.append(y)
        #zt.append(z)
        #r2.append(r2)
        print x,y,z
        
integrate_orbit('X')

<class 'astropy.units.quantity.Quantity'>
79.1747666072084 52.2277262116938 216.804562580887


TypeError: attribute of type 'int' is not callable

In [ ]:

c        -- find the final energy (the B energy is approximate at small radii)

         else if(pot.eq.'X') then
            CALL XENERGY(vcon,x,y,z,vx,vy,vz,ef(k))
        
         end if
ccc         CALL ENERGY(a,b,c,d,md,ms,vh,g,vcon,x,y,z,vx,vy,vz,ef(k))
ccc         write (6,*) 'k, e0, (e0-ef)/e0: ',k,e0(k),(e0(k)-ef(k))/e0(k)
c        --- Find the final r x v and compare
         amx=y*vz-z*vy
         amy=z*vx-x*vz
         amz=x*vy-y*vx
         am=sqrt(amx*amx+amy*amy+amz*amz)
         dbam=dbam + (dble(asin(amz/am))-bam(k))
         lamf=atan2(-amy,-amx)
         if(lamf.lt.0.0d0) lamf=lamf+2.0d0*pi
c        --- check to make sure have not crossed 0/2*pi
         if (abs(lamf-lam(k)).gt.pi) then
            if (lam(k).lt.pi/2.0d0) then
               lamf=lamf-2.0d0*pi
            else
               lamf=lamf+2.0d0*pi
            end if
         end if
         dlam=dlam + (lamf-lam(k))
c
         CALL FINDIDORB(NC,NS,NSTEP,r2,nap,idap,npe,idpe)
         if(NAP.lt.2) go to 666
c        --- Find average R_p and store it.
         rp(k)=0.0d0
         do i=1,NPE
            rp(k)=rp(k)+sqrt(r2(idpe(i)))
         end do
         rp(k)=rp(k)/real(NPE)
         if (rp(k).lt.rpval) then
            NRP=NRP+1
         end if
c        --- Find average R_a and store it.
         ra(k)=0.0d0
         do i=1,NAP
            ra(k)=ra(k)+sqrt(r2(idap(i)))
         end do
         ra(k)=ra(k)/real(NAP)
c        --- Find average period from R_a to R_a and store it.
         porb(k)=0.0d0
         do i=2,NAP
            porb(k)=porb(k)+time(idap(i))-time(idap(i-1))
         end do
         porb(k)=porb(k)/real(NAP-1)

c        --- Find average eccentricity and store it.
         e(k)=0.0d0
         do i=1,min(NPE,NAP)
            e(k)=e(k)+(sqrt(r2(idap(i)))-sqrt(r2(idpe(i))))/
     &                (sqrt(r2(idap(i)))+sqrt(r2(idpe(i))))
         end do
         e(k)=e(k)/real(min(NPE,NAP))
c        --- Calculate the tidal radius at pericenter and store it
         do i=1,min(NPE,NAP)
            rap=sqrt(r2(idap(i)))
            rpe=sqrt(r2(idpe(i)))
            ec=(rap-rpe)/(rap+rpe)
            fe=((1.00-ec)**2)/
     &         ((((1.00+ec)**2)/(2.0*ec))*dlog((1.0+ec)/(1.0-ec))+1.00)
            mmw=1.1e10*(rap+rpe)/2.0
            rt1(k)=rt1(k)+((rap+rpe)/2.0)*(fe*mol1*ldsph/mmw)**0.333333
            rt2(k)=rt2(k)+((rap+rpe)/2.0)*(fe*mol2*ldsph/mmw)**0.333333
         end do
         rt1(k)=rt1(k)/real(min(NPE,NAP))
         rt2(k)=rt2(k)/real(min(NPE,NAP))
         rt1(k)=atan(rt1(k)/dg)*ratoam
         rt2(k)=atan(rt2(k)/dg)*ratoam
         theta(k)=0.0d0
         omega(k)=0.0d0
         do i=1,min(NPE,NAP)
            CALL PICK3POINTS(NC,NS,idpe,idap,i,xt,yt,zt,p)
            CALL FINDPLANE(p,ap,bp,cp,dp)
            CALL FINDINC(ap,bp,cp,dumb)
            theta(k)=theta(k)+dumb
            CALL FINDLONGITUDE(ap,bp,dumb)
            omega(k)=omega(k)+dumb
         end do
         theta(k)=theta(k)/real(min(NPE,NAP))
         omega(k)=omega(k)/real(min(NPE,NAP))
         bpole(k)=theta(k)-(pi/2.0d0)
         lpole(k)=omega(k)+0.50d0*pi
         if (lpole(k).gt.2.0d0*pi) then
            lpole(k)=lpole(k)-2.0d0*pi
         end if
c        -- do some checking to avoid splitting omega values across 0/2pi
         if (abs(omega(k)-omega(1)).gt.pi) then
            if (omega(1).lt.pi) then
               omega(k)=omega(k)-2.0d0*pi
            else
               omega(i)=omega(k)+2.0d0*pi
            end if
         end if
c        -- do some checking to avoid splitting lpole values across 0/2pi
         if (abs(lpole(k)-lpole(1)).gt.pi) then
            if (lpole(1).lt.pi) then
               lpole(k)=lpole(k)-2.0d0*pi
            else
               lpole(k)=lpole(k)+2.0d0*pi
            end if
         end if
c        -- and the same for lam
         if (abs(lam(k)-lam(1)).gt.pi) then
            if (lam(1).lt.pi) then
               lam(k)=lam(k)-2.0d0*pi
            else
               lam(k)=lam(k)+2.0d0*pi
            end if
         end if
         k=k+1
         NCOUNT=NCOUNT+1
 666     continue
      end do
      close(unit=20)

      close(unit=1)

## Get UPGVPGWPG

In [6]:
def GETUPGVPGWPG(dlsr,vlsr,d,l,b,ug,vg,wg):
#----------------------------------------------------------------------------
#                 --- The transformation equations: ---
#
#       upg =  ug*cos(alpha) + (vg+vlsr)*sin(alpha) ;Pi direction
#       vpg = -ug*sin(alpha) + (vg+vlsr)*cos(alpha) ;Theta direction
#       wpg =  wg                                   ;Z direction
#
#     ---alpha is an angle between dlsr (a line between the Galactic center
#        and the LSR (dlsr=8500 pc) and x (a line between the Galactic center
#        and a point on the Galactic disk corresponding to a vertical 
#        projection of a galaxy onto the disk. 
#        
#     --- The circular velocity of the L.S.R. is removed.
#----------------------------------------------------------------------------


    x=np.sqrt(dlsr*dlsr+d*d*cos_deg(b)*cos_deg(b)-2.0*dlsr*d*cos_deg(b)*cos_deg(l))
    sinalpha=d*cos_deg(b)*sin_deg(l)/x
    cosalpha=(x*x+dlsr*dlsr-d*d*cos_deg(b)*cos_deg(b))/(2.0*x*dlsr)

    upg =  ug*cosalpha + (vg+vlsr)*sinalpha
    vpg = -ug*sinalpha + (vg+vlsr)*cosalpha
    wpg =  wg
    
    #upg(2) =  sqrt((ug(2)*cosalpha)**2 + (vg(2)*sinalpha)**2)
    #vpg(2) =  sqrt((ug(2)*sinalpha)**2 + (vg(2)*cosalpha)**2)
    #wpg(2) =  wg(2)
    
    return upg, vpg, wpg

upg, vpg, wpg = GETUPGVPGWPG(dlsr,vlsr,test_d,l,b,ug,vg,wg)
print upg, vpg, wpg #km/s

-36.6806351847 117.081645903 40.498068962


## Get Galactocentric tangential velocity components

In [7]:
def GETUPPGVPPGWPPG(dlsr,d,l,b,upg,vpg,wpg):
#                 --- The transformation equations: ---
#       uppg =  upg*cos(beta) + wpg*sin(beta)
#       vppg =  vpg
#       wppg = -upg*sin(beta) + wpg*cos(beta)
#
#     --- beta is an angle between the line-of-sight to a galaxy and
#         line x (see subroutine GETUPGVPGWPG for definition).
#     --- From the law of cosines...

    x=np.sqrt(dlsr*dlsr+d*d*cos_deg(b)*cos_deg(b)-2.0*dlsr*d*cos_deg(b)*cos_deg(l))
    cosbeta=x/np.sqrt(x*x+d*d*sin_deg(b)*sin_deg(b))
    sinbeta=d*sin_deg(b)/np.sqrt(x*x+d*d*sin_deg(b)*sin_deg(b))
    uppg =  upg*cosbeta + wpg*sinbeta
    vppg = vpg
    wppg = -upg*sinbeta + wpg*cosbeta

    vt = np.sqrt(vppg**2+wppg**2)
    
    return uppg, vppg, wppg, vt

GETUPPGVPPGWPPG(dlsr, 233000, l, b, upg, vpg, wpg)

(<Quantity 21.896955667014314>,
 <Quantity 117.0816459030816>,
 <Quantity 50.0608222038629>,
 <Quantity 127.33498233832428>)

### Using astropy to get XYZ position

___

## Development

In [ ]:
def xyzvelocity(ra, dec, dist, mu_a, mu_d, rv):
    pm = [mu_a, mu_d] * u.uas/(100*u.yr) 
    rv = rv * u.km/u.s
    frame = Galactocentric(z_sun=0*u.pc, galcen_distance=8.2*u.kpc)
    c = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, distance=dist*u.kpc)
    d = gc.vhel_to_gal(c.icrs, pm=pm, rv=rv, galactocentric_frame=frame, vlsr=[-usun, -vsun, wsun]*u.km/u.s)
    return d

xyzvels = xyzvelocity(test_ra, test_dec, test_d, mu_a, mu_d, rv)


print xyzvels, type(xyzvels)


## Mathematical Versions

In [ ]:
def radec_to_galactic_pos2(ra, dec, alpha_NGP=alpha_NGP, delta_NGP=delta_NGP, gc_GAN=gc_GAN):
    #Finds galactic coords (l, b) of an object given RA, DEC in degrees.       
    b = asin_deg(sin_deg(dec)*cos_deg(90.0-delta_NGP)-cos_deg(dec)*sin_deg(ra-(alpha_NGP+90.0))*sin_deg(90.0-delta_NGP))
    y = cos_deg(dec)*sin_deg(ra-(alpha_NGP+90.0))*cos_deg(90.0-delta_NGP)+sin_deg(dec)*sin_deg(90.0-delta_NGP)
    x = cos_deg(dec)*cos_deg(ra-(alpha_NGP+90.0))
    l = atan2_deg(y,x)+gc_GAN
    if l < 0:
        l += 360
    return {'l':l, 'b':b}

radec_to_galactic_pos2(test_ra, test_dec)

## Deprecated

## astropy xyz velocity

In [249]:
def xyzvelocity(ra, dec, dist, mu_a, mu_d, rv):
    pm = [mu_a, mu_d] * u.uas/(100*u.yr) 
    rv = rv * u.km/u.s
    frame = Galactocentric(z_sun=0*u.pc, galcen_distance=8.2*u.kpc)
    c = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, distance=dist*u.kpc)
    d = gc.vhel_to_gal(c.icrs, pm=pm, rv=rv, galactocentric_frame=frame, vlsr=[usun, vsun, wsun]*u.km/u.s)
    return d

xyzvels = xyzvelocity(test_ra, test_dec, test_d, mu_a, mu_d, rv)

print xyzvels, type(xyzvels)

[ -33.42046638  211.14060882   79.89657993] km / s <class 'astropy.units.quantity.Quantity'>


Numpy matrix instances are no longer used to represent rotation matrices, since
they do not allow one to represent stacks of matrices. Instead, plain arrays
are used. Instead of %(func)s, use %(alternative)s.
For matrix multiplication and tranposes, it is suggested to use
:func:`~astropy.coordinates.matrix_utilities.matrix_product` and
:func:`~astropy.coordinates.matrix_utilities.matrix_transpose`, respectively.
 [astropy.utils.decorators]
Numpy matrix instances are no longer used to represent rotation matrices, since
they do not allow one to represent stacks of matrices. Instead, plain arrays
are used. Instead of %(func)s, use %(alternative)s.
For matrix multiplication and tranposes, it is suggested to use
:func:`~astropy.coordinates.matrix_utilities.matrix_product` and
:func:`~astropy.coordinates.matrix_utilities.matrix_transpose`, respectively.
 [astropy.utils.decorators]
Numpy matrix instances are no longer used to represent rotation matrices, since
they do not allow one to represe